In [1]:
# Simplified imports for single GPU
from logging import getLogger
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from memory_layers import HashingMemory, MemoryLayerMonitorAndCheckpoint, load_and_process_dataset, ModelEvaluator

logger = getLogger()

/home/sagoyal/research/finetuningmemorylayers/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load Qwen0.5 Instruct
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", dtype=torch.float16)

# Qwen0.5 specs: 896 hidden_dim, 24 layers
hidden_dim = 896
layers_to_replace = [6, 12, 18]  # Which FFN layers to replace

# Replace FFNs with Memory Layers
for layer_idx in layers_to_replace:
    layer = model.model.layers[layer_idx]
    
    # Create memory layer
    memory_layer = HashingMemory(
        input_dim=hidden_dim,
        output_dim=hidden_dim,
        mem_n_keys=128,          # Memory size = 512² = 262k entries
        mem_heads=4,
        mem_knn=16,
        mem_k_dim=256,
        mem_v_dim=-1,            # Auto: uses output_dim
        swilu_projection=True,
        value_fixed_lr=0.001,
        mem_share_values=False,  # Don't share across layers for fine-tuning
    )
    
    # Initialize the memory layer
    memory_layer.reset_parameters()
    # Ensure memory layer matches model dtype (float16)
    memory_layer.to(device)
    
    # Replace the FFN (MLP) with memory layer
    original_mlp = layer.mlp
    layer.mlp = memory_layer
    
    print(f"Replaced layer {layer_idx} FFN with memory layer")

# FREEZE EVERYTHING EXCEPT MEMORY LAYERS
for name, param in model.named_parameters():
    if 'mlp' in name and any(f'layers.{idx}.' in name for idx in layers_to_replace):
        # This is a memory layer parameter - keep trainable
        param.requires_grad = True
        print(f"✓ Trainable: {name}")
    else:
        # Freeze all other parameters
        param.requires_grad = False

# Verify what's trainable
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTrainable: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.2f}%)")

Replaced layer 6 FFN with memory layer
Replaced layer 12 FFN with memory layer
Replaced layer 18 FFN with memory layer
✓ Trainable: model.layers.6.mlp.keys
✓ Trainable: model.layers.6.mlp.values.weight
✓ Trainable: model.layers.6.mlp.value_proj.weight
✓ Trainable: model.layers.6.mlp.value_proj.bias
✓ Trainable: model.layers.6.mlp.swilu_projection.weight
✓ Trainable: model.layers.6.mlp.swilu_projection.bias
✓ Trainable: model.layers.6.mlp.query_proj.query_mlps.0.weight
✓ Trainable: model.layers.6.mlp.query_proj.query_mlps.0.bias
✓ Trainable: model.layers.12.mlp.keys
✓ Trainable: model.layers.12.mlp.values.weight
✓ Trainable: model.layers.12.mlp.value_proj.weight
✓ Trainable: model.layers.12.mlp.value_proj.bias
✓ Trainable: model.layers.12.mlp.swilu_projection.weight
✓ Trainable: model.layers.12.mlp.swilu_projection.bias
✓ Trainable: model.layers.12.mlp.query_proj.query_mlps.0.weight
✓ Trainable: model.layers.12.mlp.query_proj.query_mlps.0.bias
✓ Trainable: model.layers.18.mlp.keys
✓ Tra

In [3]:
# Load and process dataset
tokenized = load_and_process_dataset(tokenizer, sample_size=10000)

Filtered dataset size: 7669
Tokenized dataset: Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 7669
})


In [4]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from memory_layers import ModelEvaluator

# Training arguments optimized for memory layers only
training_args = TrainingArguments(
    output_dir="./qwen_memory_finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-4,  # Higher LR since only training memory
    warmup_steps=100,
    lr_scheduler_type="cosine",
    logging_steps=10,
    logging_first_step=True,  # Log immediately
    logging_dir="./logs",
    save_steps=500,
    eval_strategy="steps",
    eval_steps=100,   
    # Performance
    fp16=True,
    gradient_checkpointing=False,  # Not needed with frozen base
    dataloader_num_workers=2,
    
    # Monitoring
    report_to="tensorboard",  # or "wandb" if you have it
    # load_best_model_at_end=True,
    metric_for_best_model="loss",
    save_strategy="no",
    
    # Memory optimization
    optim="adamw_torch_fused",  # Faster optimizer
    max_grad_norm=1.0,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize Evaluator
evaluator = ModelEvaluator(model, tokenizer, device=device)

# Initialize callback
memory_monitor = MemoryLayerMonitorAndCheckpoint(
    model=model,
    layers_to_check=layers_to_replace,
    save_every=500,
    keep_last=2,
    monitor_every=50,
    evaluator=evaluator,
    eval_every=50,     # Run evaluation every 100 steps
    eval_samples=20     # Small sample size for speed during training
)

# Create trainer with callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized.select(range(1000)),  # Use 1k for validation
    data_collator=data_collator,
    callbacks=[memory_monitor],  # Add our custom monitor
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
print("\n🚀 Starting training...")
print(f"Total steps: {len(tokenized) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

# Train! 
trainer.train()

print("\n✅ Training complete!")


🚀 Starting training...
Total steps: 1437


Step,Training Loss,Validation Loss
100,2.011300,1.947128
200,1.845300,1.891734
300,1.915300,1.855640
400,1.846400,1.823111
500,1.714400,1.788283
600,1.801700,1.745393
700,1.896200,1.716421
800,1.869000,1.689151
900,1.763000,1.666582
1000,1.623100,1.637899


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'trivia_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.



🔍 MEMORY LAYER HEALTH CHECK - Step 50

📊 Layer 6 Memory:
  Parameters:
    Keys:   mean=-0.0001, std=0.0361
    Values: mean=-0.0000, std=0.0334
  Changes since start:
    Keys:   0.000883 ✅
    Values: 0.000788 ✅

📊 Layer 12 Memory:
  Parameters:
    Keys:   mean=-0.0000, std=0.0361
    Values: mean=+0.0000, std=0.0334
  Changes since start:
    Keys:   0.000818 ✅
    Values: 0.000778 ✅

📊 Layer 18 Memory:
  Parameters:
    Keys:   mean=+0.0000, std=0.0361
    Values: mean=-0.0000, std=0.0334
  Changes since start:
    Keys:   0.000802 ✅
    Values: 0.000746 ✅

✅ All memory layers healthy!


📊 RUNNING BENCHMARK EVALUATION - Step 50
Evaluating on TriviaQA (validation, 20 samples)...


100%|██████████| 20/20 [00:15<00:00,  1.25it/s]


...


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'trivia_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.



🔍 MEMORY LAYER HEALTH CHECK - Step 1400

📊 Layer 6 Memory:
  Parameters:
    Keys:   mean=+0.0001, std=0.0385
    Values: mean=-0.0000, std=0.0340
  Changes since start:
    Keys:   0.007515 ✅
    Values: 0.003739 ✅

📊 Layer 12 Memory:
  Parameters:
    Keys:   mean=-0.0000, std=0.0385
    Values: mean=+0.0001, std=0.0339
  Changes since start:
    Keys:   0.007828 ✅
    Values: 0.003244 ✅

📊 Layer 18 Memory:
  Parameters:
    Keys:   mean=+0.0001, std=0.0401
    Values: mean=-0.0000, std=0.0344
  Changes since start:
    Keys:   0.009232 ✅
    Values: 0.005126 ✅

✅ All memory layers healthy!


📊 RUNNING BENCHMARK EVALUATION - Step 1400
Evaluating on TriviaQA (validation, 20 samples)...


100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


TriviaQA Accuracy: 15.00%
Evaluating on GSM8K (test, 20 samples)...


 35%|███▌      | 7/20 [00:44<01:23,  6.41s/it]

In [ ]:
# 1. Check actual loss history
import pandas as pd

# Extract all losses from trainer state
losses = [(log.get('step', i), log.get('loss'), log.get('eval_loss')) 
          for i, log in enumerate(trainer.state.log_history)]

df = pd.DataFrame(losses, columns=['step', 'train_loss', 'eval_loss'])
print(df)

# 2. Check if model is producing valid outputs
model.eval()
test_input = tokenizer("Hello, how are you?", return_tensors="pt").to(device)


    step  train_loss  eval_loss
0      1      2.0335        NaN
1     10      2.1182        NaN
2     20      2.0718        NaN
3     30      2.1551        NaN
4     40      7.7784        NaN
5     50    254.0208        NaN
6     60    280.1049        NaN
7     70    246.2023        NaN
8     80      0.0000        NaN
9     90      0.0000        NaN
10   100      0.0000        NaN
11   100         NaN        NaN
12   110      0.0000        NaN
13   120      0.0000        NaN
14   130      0.0000        NaN
15   140      0.0000        NaN


In [ ]:
with torch.no_grad():
    outputs = model(**test_input)
    logits = outputs.logits
    
    print(f"\nLogits stats:")
    print(f"  Mean: {logits.mean().item()}")
    print(f"  Std: {logits.std().item()}")
    print(f"  Min: {logits.min().item()}")
    print(f"  Max: {logits.max().item()}")
    print(f"  Any NaN: {torch.isnan(logits).any().item()}")
    print(f"  Any Inf: {torch.isinf(logits).any().item()}")

# 3. Generate some text to see if model is working
generated = model.generate(**test_input, max_new_tokens=20)
print(f"\nGenerated text: {tokenizer.decode(generated[0])}")


Logits stats:
  Mean: nan
  Std: nan
  Min: nan
  Max: nan
  Any NaN: True
  Any Inf: False


AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


/pytorch/aten/src/ATen/native/cuda/TensorCompare.cu:112: _assert_async_cuda_kernel: block: [0,0,0], thread: [0,0,0] Assertion `probability tensor contains either `inf`, `nan` or element < 0` failed.


In [ ]:
# Get all logged history
log_history = trainer.state.log_history

# Extract just the loss entries
losses = [log['loss'] for log in log_history if 'loss' in log]
steps = [log['step'] for log in log_history if 'loss' in log]

# Plot
plt.plot(steps, losses)
plt.show()

In [ ]:
from safetensors.torch import load_file

device = "cuda" if torch.cuda.is_available() else "cpu"
hidden_dim = 896
layers_to_replace = [6, 12, 18]

# Reload model for testing
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    dtype=torch.float16,
).to(device)

# Add memory layers
for idx in layers_to_replace:
    # Initialize and cast to correct device/dtype
    mem_layer = HashingMemory(
        input_dim=hidden_dim, output_dim=hidden_dim, mem_n_keys=128, mem_heads=4,
        mem_knn=16, mem_k_dim=256, mem_v_dim=-1, swilu_projection=True,
        value_fixed_lr=0.001, mem_share_values=False
    )
    # Important: Cast to model's dtype (float16) to avoid "Half and Float" errors
    model.model.layers[idx].mlp = mem_layer.to(device, dtype=model.dtype)

# Load weights
try:
    state_dict = load_file("./qwen_memory_final/model.safetensors")
except:
    state_dict = torch.load("./qwen_memory_final/pytorch_model.bin", 
                           weights_only=False)

model.load_state_dict(state_dict, strict=False)
print("\n✅ Model loaded successfully!")

# Test generation
def test_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Try some prompts
# test_prompts = [
#     "Explain quantum computing in simple terms:",
#     "Write a Python function to sort a list:",
#     "What are the health benefits of exercise?",
# ]

# for prompt in test_prompts:
#     print(f"\n{'='*80}")
#     print(f"Prompt: {prompt}")
#     print(f"{'='*80}")
#     response = test_model(prompt)
#     print(response)


✅ Model loaded successfully!


In [ ]:
# Load original Qwen model for comparison
base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    torch_dtype=torch.float16,
)
base_model.to(device)

def compare_models(prompt):
    # Your fine-tuned model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        # Fine-tuned
        ft_outputs = model.generate(**inputs, max_new_tokens=100)
        ft_response = tokenizer.decode(ft_outputs[0], skip_special_tokens=True)
        
        # Base
        base_outputs = base_model.generate(**inputs, max_new_tokens=100)
        base_response = tokenizer.decode(base_outputs[0], skip_special_tokens=True)
    
    print(f"\n{'='*80}")
    print(f"Prompt: {prompt}")
    print(f"{'='*80}")
    print(f"\n🔷 BASE MODEL:")
    print(base_response)
    print(f"\n🔶 FINE-TUNED (with memory layers):")
    print(ft_response)
    print(f"{'='*80}\n")

# Test
compare_models("Explain machine learning:")